In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad

import scdrs.data_loader as dl

In [7]:
DATA_PATH='/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data'
df_hom = pd.read_csv('/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gene_annotation/'
                     'mouse_human_homologs.txt', sep='\t')
dic_map = {x:y for x,y in zip(df_hom['HUMAN_GENE_SYM'], df_hom['MOUSE_GENE_SYM'])}
dic_map_m2h = {x:y for x,y in zip(df_hom['MOUSE_GENE_SYM'], df_hom['HUMAN_GENE_SYM'])}

In [8]:
# Richtor 2021 
dic_gs = {}
adata = dl.load_richter(DATA_PATH, opt='raw', flag_size_factor=True, flag_log1p=True)
adata = adata[adata.obs['inferred_cell_label']=='Hepatocytes'].copy()
adata = adata[:,[x for x in adata.var_names if x.startswith('ERCC') is False]]

sc.tl.rank_genes_groups(adata, groupby='Ploidy', method='wilcoxon')

temp_df = sc.get.rank_genes_groups_df(adata, group='4n')
ind_select = (temp_df['logfoldchanges']>0.5) & (temp_df['pvals_adj']<0.01)
print('4n', ind_select.sum())
dic_gs['hep4n_richter'] = [dic_map_m2h[x] for x in temp_df.loc[ind_select, 'names'][:200] 
                           if x in dic_map_m2h]
ind_select = (temp_df['logfoldchanges']<-0.1) & (temp_df['pvals_adj']<0.01)
print('2n', ind_select.sum())
dic_gs['hep2n_richter'] = [dic_map_m2h[x] for x in temp_df['names'][-200:] 
                           if x in dic_map_m2h]

Trying to set attribute `.uns` of view, copying.


4n 830
2n 5


In [11]:
# Table S1 in Miettinen CurrBio 2014

temp_df = pd.read_excel('/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/single_cell_data/'
                        'miettinen_CurrBio_2014/Miettinen_CurrBio_2014_supp_tab.xlsx', skiprows=4)

# ind_select = (temp_df['Pearson correlation with nuclear radius ( r )']>0.8) & \
#                 (temp_df['correlation adj.P.val']<0.01)
# dic_gs['size_pos_meittinen'] = [dic_map_m2h[x] for x in temp_df.loc[ind_select, 'Gene_Name']
#                                 if x in dic_map_m2h] 

# ind_select = (temp_df['Pearson correlation with nuclear radius ( r )']<-0.8) & \
#                 (temp_df['correlation adj.P.val']<0.01)
# dic_gs['size_neg_meittinen'] = [dic_map_m2h[x] for x in temp_df.loc[ind_select, 'Gene_Name']
#                                 if x in dic_map_m2h] 

# # 
# ind_select = (temp_df['[Cdk1Flox/Flox_Pre-PH] vs [Cdk1Liv-/-_Pre-PH] logFC']>0.5) & \
#                 (temp_df['[Cdk1Flox/Flox_Pre-PH] vs [Cdk1Liv-/-_Pre-PH] adj.P.val']<0.01)
# dic_gs['ctrl_vs_case_prePH'] = [dic_map_m2h[x] for x in temp_df.loc[ind_select, 'Gene_Name']
#                                 if x in dic_map_m2h]

# ind_select = (temp_df['[Cdk1Flox/Flox_Pre-PH] vs [Cdk1Liv-/-_Pre-PH] logFC']<-0.5) & \
#                 (temp_df['[Cdk1Flox/Flox_Pre-PH] vs [Cdk1Liv-/-_Pre-PH] adj.P.val']<0.01)
# dic_gs['case_vs_ctrl_prePH'] = [dic_map_m2h[x] for x in temp_df.loc[ind_select, 'Gene_Name']
#                                 if x in dic_map_m2h]

# 
ind_select = (temp_df['[Cdk1Flox/Flox_96hPH] vs [Cdk1Liv-/-_96hPH] logFC']>0.5) & \
                (temp_df['[Cdk1Flox/Flox_96hPH] vs [Cdk1Liv-/-_96hPH] adj.P.val']<0.01)
dic_gs['ctrl_vs_case_96hPH'] = [dic_map_m2h[x] for x in temp_df.loc[ind_select, 'Gene_Name']
                                if x in dic_map_m2h]

ind_select = (temp_df['[Cdk1Flox/Flox_96hPH] vs [Cdk1Liv-/-_96hPH] logFC']<-0.5) & \
                (temp_df['[Cdk1Flox/Flox_96hPH] vs [Cdk1Liv-/-_96hPH] adj.P.val']<0.01)
dic_gs['case_vs_ctrl_96hPH'] = [dic_map_m2h[x] for x in temp_df.loc[ind_select, 'Gene_Name'] 
                               if x in dic_map_m2h] 



# 
ind_select = (temp_df['[Cdk1Liv-/-_Pre-PH] vs [Cdk1Liv-/-_96hPH] logFC']>0.5) & \
                (temp_df['[Cdk1Liv-/-_Pre-PH] vs [Cdk1Liv-/-_96hPH] adj.P.val']<0.01)
dic_gs['case_prePH_vs_96hPH'] = [dic_map_m2h[x] for x in temp_df.loc[ind_select, 'Gene_Name'] 
                                 if x in dic_map_m2h]

ind_select = (temp_df['[Cdk1Liv-/-_Pre-PH] vs [Cdk1Liv-/-_96hPH] logFC']<-0.5) & \
                (temp_df['[Cdk1Liv-/-_Pre-PH] vs [Cdk1Liv-/-_96hPH] adj.P.val']<0.01)
dic_gs['case_96hPH_vs_prePH'] = [dic_map_m2h[x] for x in temp_df.loc[ind_select, 'Gene_Name'] 
                                 if x in dic_map_m2h]

dic_gs['large_vs_small'] = temp_df.sort_values('log2FoldChange(Largest vs smallest)')['Gene_Name'][:500]
dic_gs['large_vs_small'] = [dic_map_m2h[x] for x in dic_gs['large_vs_small'] 
                            if x in dic_map_m2h] 

In [12]:
for term in dic_gs:
    print(term, len(dic_gs[term]), 
          'overlap with hep4n_richtor', len(set(dic_gs[term]) & set(dic_gs['hep4n_richter'])),
          'overlap with hep2n_richtor', len(set(dic_gs[term]) & set(dic_gs['hep2n_richter'])))

hep4n_richter 144 overlap with hep4n_richtor 144 overlap with hep2n_richtor 0
hep2n_richter 134 overlap with hep4n_richtor 0 overlap with hep2n_richtor 134
ctrl_vs_case_96hPH 509 overlap with hep4n_richtor 1 overlap with hep2n_richtor 12
case_vs_ctrl_96hPH 399 overlap with hep4n_richtor 30 overlap with hep2n_richtor 7
case_prePH_vs_96hPH 784 overlap with hep4n_richtor 3 overlap with hep2n_richtor 20
case_96hPH_vs_prePH 472 overlap with hep4n_richtor 39 overlap with hep2n_richtor 5
large_vs_small 261 overlap with hep4n_richtor 16 overlap with hep2n_richtor 1


In [18]:
df_gs = pd.DataFrame(columns=['TRAIT', 'GENESET'])
for term in dic_gs:
    df_gs.loc[term] = [term, ','.join(dic_gs[term])]
df_gs.to_csv('/n/holystore01/LABS/price_lab/Users/mjzhang/scTRS_data/gs_file/ploidy.gs',
             sep='\t', index=False)

### Old code 